In [1]:
import os
import pandas as pd
from datetime import datetime as dt
import time
import plotly.express as px
import mysql.connector
from mysql.connector import Error
import numpy as np
import config

# Connecting to Wordpress Database

In [2]:
host = config.host
database = config.database
user = config.user
password = config.password

conn = mysql.connector.connect(host = host, database = database, user = user, password = password)

# Wordpress wp_jet_appointment table

#### SQL to query appointments

In [3]:
appointments = "SELECT user_id, status, service, provider, user_email, date, slot, slot_end \
                FROM wp_jet_appointments \
                ORDER BY slot asc"

#### Reading sql query and saving to csv

In [4]:
ap_data = pd.read_sql_query(appointments, conn)
ap_data.to_csv("assets/data/wp_jet_appointments.csv", index=False)

#### Loading Appointment Data from csv

In [5]:
ap = pd.read_csv("assets/data/wp_jet_appointments.csv")
ap

,user_id,status,service,provider,user_email,date,slot,slot_end
0,NaN,pending,404,415,abrahammrjones@gmail.com,1591833600,1591876800,1591880400
1,10.0,pending,456,417,abrahammrjones@gmail.com,1591833600,1591880400,1591891200
2,10.0,pending,460,415,abrahammrjones@gmail.com,1591833600,1591880400,1591884000
3,10.0,pending,458,415,abrahammrjones@gmail.com,1591833600,1591884000,1591891200
4,10.0,pending,448,415,abrahammrjones@gmail.com,1591833600,1591884000,1591887600
5,10.0,pending,442,415,abrahammrjones@gmail.com,1591833600,1591902000,1591905600


## Calculating duration times

In [6]:
duration = pd.to_datetime((ap.slot_end - ap.slot), unit='s').dt.time

#### Adding duration column to appointment dataframe

In [7]:
ap["duration"] = duration
ap

,user_id,status,service,provider,user_email,date,slot,slot_end,duration
0,NaN,pending,404,415,abrahammrjones@gmail.com,1591833600,1591876800,1591880400,01:00:00
1,10.0,pending,456,417,abrahammrjones@gmail.com,1591833600,1591880400,1591891200,03:00:00
2,10.0,pending,460,415,abrahammrjones@gmail.com,1591833600,1591880400,1591884000,01:00:00
3,10.0,pending,458,415,abrahammrjones@gmail.com,1591833600,1591884000,1591891200,02:00:00
4,10.0,pending,448,415,abrahammrjones@gmail.com,1591833600,1591884000,1591887600,01:00:00
5,10.0,pending,442,415,abrahammrjones@gmail.com,1591833600,1591902000,1591905600,01:00:00


## Convert data series to DateTime formatting

In [8]:
# Using a unix epoch time
ap["date"] = pd.to_datetime(ap["date"], unit='s')
ap["slot"] = pd.to_datetime(ap["slot"], unit='s').dt.time
ap["slot_end"] = pd.to_datetime(ap["slot_end"], unit='s').dt.time
ap

,user_id,status,service,provider,user_email,date,slot,slot_end,duration
0,NaN,pending,404,415,abrahammrjones@gmail.com,2020-06-11,12:00:00,13:00:00,01:00:00
1,10.0,pending,456,417,abrahammrjones@gmail.com,2020-06-11,13:00:00,16:00:00,03:00:00
2,10.0,pending,460,415,abrahammrjones@gmail.com,2020-06-11,13:00:00,14:00:00,01:00:00
3,10.0,pending,458,415,abrahammrjones@gmail.com,2020-06-11,14:00:00,16:00:00,02:00:00
4,10.0,pending,448,415,abrahammrjones@gmail.com,2020-06-11,14:00:00,15:00:00,01:00:00
5,10.0,pending,442,415,abrahammrjones@gmail.com,2020-06-11,19:00:00,20:00:00,01:00:00


#### Converting 24 hours to 12 hours AM/PM

In [9]:
ap["slot"] = ap["slot"].apply(lambda x: x.strftime('%I:%M %p'))
ap["slot_end"] = ap["slot_end"].apply(lambda x: x.strftime('%I:%M %p'))
ap

,user_id,status,service,provider,user_email,date,slot,slot_end,duration
0,NaN,pending,404,415,abrahammrjones@gmail.com,2020-06-11,12:00 PM,01:00 PM,01:00:00
1,10.0,pending,456,417,abrahammrjones@gmail.com,2020-06-11,01:00 PM,04:00 PM,03:00:00
2,10.0,pending,460,415,abrahammrjones@gmail.com,2020-06-11,01:00 PM,02:00 PM,01:00:00
3,10.0,pending,458,415,abrahammrjones@gmail.com,2020-06-11,02:00 PM,04:00 PM,02:00:00
4,10.0,pending,448,415,abrahammrjones@gmail.com,2020-06-11,02:00 PM,03:00 PM,01:00:00
5,10.0,pending,442,415,abrahammrjones@gmail.com,2020-06-11,07:00 PM,08:00 PM,01:00:00


## Replacing provider ID with provider name

In [10]:
ap["provider"] = ap["provider"].replace(415, "City Winery at the Chicago Riverwalk")
ap["provider"] = ap["provider"].replace(417, "Marina City")
ap["provider"] = ap["provider"].replace(421, "Pizzeria Portofino")
ap

,user_id,status,service,provider,user_email,date,slot,slot_end,duration
0,NaN,pending,404,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,12:00 PM,01:00 PM,01:00:00
1,10.0,pending,456,Marina City,abrahammrjones@gmail.com,2020-06-11,01:00 PM,04:00 PM,03:00:00
2,10.0,pending,460,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,01:00 PM,02:00 PM,01:00:00
3,10.0,pending,458,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,02:00 PM,04:00 PM,02:00:00
4,10.0,pending,448,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,02:00 PM,03:00 PM,01:00:00
5,10.0,pending,442,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,07:00 PM,08:00 PM,01:00:00


## Replacing service Id with name

In [11]:
ap["service"] = ap["service"].replace(478, "Dockage Under 24' 1hr")
ap["service"] = ap["service"].replace(476, "Dockage Under 24' 2hr")
ap["service"] = ap["service"].replace(474, "Dockage Under 24' 3hr")
ap["service"] = ap["service"].replace(472, "Dockage 25' to 34' 1hr")
ap["service"] = ap["service"].replace(470, "Dockage 25' to 34' 2hr")
ap["service"] = ap["service"].replace(468, "Dockage 25' to 34' 3hr")
ap["service"] = ap["service"].replace(466, "Dockage 35' to 44' 1hr")
ap["service"] = ap["service"].replace(464, "Dockage 35' to 44' 2hr")
ap["service"] = ap["service"].replace(462, "Dockage 35' to 44' 3hr")
ap["service"] = ap["service"].replace(460, "Dockage 45' to 54' 1hr")
ap["service"] = ap["service"].replace(458, "Dockage 45' to 54' 2hr")
ap["service"] = ap["service"].replace(456, "Dockage 45' to 54' 3hr")
ap["service"] = ap["service"].replace(454, "Dockage 55' to 64' 1hr")
ap["service"] = ap["service"].replace(452, "Dockage 55' to 64' 2hr")
ap["service"] = ap["service"].replace(450, "Dockage 55' to 64' 3hr")
ap["service"] = ap["service"].replace(448, "Dockage 65' to 124' 1hr")
ap["service"] = ap["service"].replace(446, "Dockage 65' to 124' 2hr")
ap["service"] = ap["service"].replace(444, "Dockage 65' to 124' 3hr")
ap["service"] = ap["service"].replace(442, "Dockage 125' and more 1hr")
ap["service"] = ap["service"].replace(440, "Dockage 125' and more 2hr")
ap["service"] = ap["service"].replace(438, "Dockage 125' and more 3hr")
ap

,user_id,status,service,provider,user_email,date,slot,slot_end,duration
0,NaN,pending,404,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,12:00 PM,01:00 PM,01:00:00
1,10.0,pending,Dockage 45' to 54' 3hr,Marina City,abrahammrjones@gmail.com,2020-06-11,01:00 PM,04:00 PM,03:00:00
2,10.0,pending,Dockage 45' to 54' 1hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,01:00 PM,02:00 PM,01:00:00
3,10.0,pending,Dockage 45' to 54' 2hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,02:00 PM,04:00 PM,02:00:00
4,10.0,pending,Dockage 65' to 124' 1hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,02:00 PM,03:00 PM,01:00:00
5,10.0,pending,Dockage 125' and more 1hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,07:00 PM,08:00 PM,01:00:00


# Wordpress wp_usermeta table

#### SQL to query wp_usermeta

In [12]:
user = "SELECT * \
        FROM wp_usermeta \
        WHERE meta_key = 'length' "

#### Reading sql query and saving to csv

In [13]:
usermeta_data = pd.read_sql_query(user, conn)
usermeta_data.to_csv("assets/data/wp_usermeta.csv", index=False)

#### Loading Appointment Data from csv

In [14]:
user = pd.read_csv("assets/data/wp_usermeta.csv")
user

,umeta_id,user_id,meta_key,meta_value
0,194,10,length,75
